In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import logging
import time
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode (without opening browser window)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Create a new WebDriver instance



In [30]:
def scrapFromLocation(speciality, location):
    # Open the URL
    driver = webdriver.Chrome(options=chrome_options)
    speciality.replace(' ', '%20')
    location.replace(' ', '%20')
    driver.get(f'https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22{speciality}%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%2C%7B%22word%22%3A%22{location}%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22locality%22%7D%5D&city=Bangalore')
    # Get page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Keep scrolling to the bottom of the page until no more content is loaded
    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for some time to allow content to load
        time.sleep(2)  # Adjust delay as needed
        last_height = driver.execute_script("return document.body.scrollHeight")
        # Check if the page height has changed after scrolling
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Get the updated page source after scrolling
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Navigate to the container with all doctor cards
    doctor_cards = soup.find_all('div', class_='u-border-general--bottom')

    intro='https://www.practo.com/'
    urls=[]
    # Extract URLs from each card
    for card in doctor_cards:
        try:
            # Try to find the div containing the <a> tag
            div_tag = card.find('div', class_='info-section')
            if div_tag:
                # Within the div, find the <a> tag
                anchor_tag = div_tag.find('a', href=True)
                if anchor_tag:
                    url = anchor_tag['href']
                    urls.append(intro+url)
                else:
                    logging.info("URL not found within the div.")
            else:
                logging.info("Div tag not found in the card.")

            
        except AttributeError as e:
            logging.info(f"AttributeError: {e}")
    logging.info(f"Found {urls} URLs.")
    # Close the WebDriver
    driver.quit()
    return urls

In [31]:
urls=scrapFromLocation('Dentist','Yelahanka')

In [32]:
len(urls)

16